# Clip Basic Data (DEM, DDM, FAM)
We use data from Hydrosheds with 3s resolution (approx 90m at the equator). The DEM already conditioned for hydrology (pit removed, sink filled, stream burned). The flow direction already in ESRI DDM, and flow accumulation also ready to use https://www.hydrosheds.org/hydrosheds-core-downloads. Here we need to clip the data to our area of interest and set the format to Float32

In [22]:
import rioxarray
import numpy as np

def clip_raster(src_raster, dst_raster, extent):
    da = rioxarray.open_rasterio(src_raster, masked=False).squeeze()

    xmin, ymin, xmax, ymax = extent
    da_clipped = da.rio.clip_box(minx=xmin, miny=ymin, maxx=xmax, maxy=ymax)

    da_clipped = da_clipped.astype("float32")

    da_clipped.rio.to_raster(dst_raster, compress="LZW", tiled=False)
    
    return da_clipped

extent = (105.0, -8.0, 108.9, -5.7)   # xmin, ymin, xmax, ymax
da = clip_raster(
    src_raster="D:/Data/EF5/soil_param_b_5km_global_corrected.tif",
    dst_raster="soil_param_b_5km_global_jabar_float32.tif",
    extent=extent,
)

CPLE_AppDefinedError: Deleting soil_param_b_5km_global_jabar_float32.tif failed: Permission denied

# Align CREST grid parameters

EF5 requires the gridded CREST parameter to be at the exact same grid resolution and extent as the DEM. Hence, we need to regrid the parameters and clip them to the same extent as the DEM. First, some parameter grid is off by several degrees to southwest (you need to check on QGIS for this), we will fix that by adjusting the data.

In [2]:
import rasterio
import numpy as np
from rasterio.enums import Resampling

# === Parameter shift (dalam jumlah pixel) ===
row_shift = -4   # negative = shift north, positive = shift south
col_shift = 23   # negative = shift west, positive = shift east

# === Input/Output ===
input_file = "D:/Data/EF5/soil_param_b_5km_global.tif"      # shifted raster
output_file = "D:/Data/EF5/soil_param_b_5km_global_corrected.tif"

# === Baca raster input ===
with rasterio.open(input_file) as src:
    profile = src.profile
    data = src.read(1)
    nodata = src.nodata if src.nodata is not None else -9999

# === Geser array ===
corrected = np.full_like(data, nodata)  # isi awal dengan nodata

nrows, ncols = data.shape

# Hitung indeks sumber dan target
row_idx_src = np.arange(nrows)
col_idx_src = np.arange(ncols)

row_idx_tgt = row_idx_src + row_shift
col_idx_tgt = col_idx_src + col_shift

# Filter supaya tetap dalam batas array
valid_rows = (row_idx_tgt >= 0) & (row_idx_tgt < nrows)
valid_cols = (col_idx_tgt >= 0) & (col_idx_tgt < ncols)

# Copy isi pixel
for i_src, i_tgt in zip(row_idx_src[valid_rows], row_idx_tgt[valid_rows]):
    for j_src, j_tgt in zip(col_idx_src[valid_cols], col_idx_tgt[valid_cols]):
        corrected[i_tgt, j_tgt] = data[i_src, j_src]

# === Simpan hasil ===
profile.update(dtype=rasterio.float32, compress="DEFLATE", nodata=nodata)

with rasterio.open(output_file, "w", **profile) as dst:
    dst.write(corrected.astype(np.float32), 1)

print(f"Hasil disimpan ke {output_file}")

Hasil disimpan ke D:/Data/EF5/soil_param_b_5km_global_corrected.tif


Now we can align the grid using xarray interpolation

In [3]:
import rioxarray

In [44]:
dem_file = "dem_clipped_jabar_float32.tif"
param_file = "D:/Data/EF5/soil_param_corrected/soil_param_wm_5km_global_corrected_jabar.tif"

In [45]:
da_dem = rioxarray.open_rasterio(dem_file, masked=True).squeeze()
da_input = rioxarray.open_rasterio(param_file, masked=True).squeeze()
dem_lats = da_dem.y.values
dem_lons = da_dem.x.values

#interpolate using 'nearest' method
da_interp = da_input.interp(x=dem_lons, y=dem_lats, method="nearest")

In [46]:
da_interp.rio.to_raster(
    "D:/Projects/ef5-preprocessing/data/parameters/CREST/soil_param_wm_5km_global_corrected_jabar.tif",
    compress="LZW",
    tiled=False,
)

## Automate Gauge Placement
Gauge in EF5 can be a virtual or real gauge. In this case we will place virtual gauge based on administration boundaries. We use the admin boundary and flow accumulation data. The idea is to place gauge on the highest flow accumulation pixel in an admin boundary.

In [10]:
import rioxarray
import geopandas as gpd
import os
import numpy as np

In [6]:
#Flow Accumulation Data
fam = rioxarray.open_rasterio("D:/Projects/ef5-preprocessing/data/basic/fam_clipped_jabar_float32.tif", masked=True).squeeze()

#Administration boundary data
kel = gpd.read_file("D:/Data/geojson_kelurahan/31/kelurahan_jakarta_all.geojson")
kel = kel.to_crs("EPSG:4326")

In [8]:
#Empty dictionary to hold extracted data
data = {
    'name':[],
    'fam':[],
    'lon':[],
    'lat':[],
}

In [11]:
#Iterate through all geodataframe
for i, row in kel.iterrows():
    name = row['NAMOBJ']
    geom = row['geometry']
    try:
        #clip flow accumulation based on boundary
        fam_clip = fam.rio.clip([geom], kel.crs, drop=True, all_touched=False)
        #looking for maximum fam value
        argmax_flat = fam_clip.argmax().item()   # posisi linear
        y_ind, x_ind = np.unravel_index(argmax_flat, fam_clip.shape)
    
        # convert from index to coordinate
        lon = fam_clip["x"].values[x_ind]
        lat = fam_clip["y"].values[y_ind]
    
        data['name'].append(name)
        data['fam'].append(fam_clip.max().item())
        data['lon'].append(lon)
        data['lat'].append(lat)
    except Exception as e:
        print(f"Error processing {name}: {e}")
        data['name'].append(name)
        data['fam'].append(None)
        data['lon'].append(None)
        data['lat'].append(None)

Error processing Pulau Kelapa: No data found in bounds.
Error processing Pulau Harapan: No data found in bounds.


In [12]:
#transform into geodataframe and save to geojson
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['lon'], data['lat']), crs="EPSG:4326")
gdf = gdf.dropna(how='any', subset=['lon', 'lat'])
gdf.to_file("gauge_potential_jakarta.geojson", driver='GeoJSON')

You need to validate the location later in QGIS to make sure the gauge is correctly placed in the "river"

## Generate Gauge and Basin
This part is just to automate the generation of "GAUGE" and "BASIN" part, you need to copy and paste the output to the original control file.

In [13]:
# Konversi gauge location to config
import json

# --- Muat GeoJSON ---
with open("gauge_potential_jakarta.geojson") as f:
    geojson = json.load(f)

output_file = "gauges_jakarta.ini"

with open(output_file, "w") as f:
    for feature in geojson["features"]:
        name = feature["properties"]["name"]
        lon, lat = feature["geometry"]["coordinates"]
        f.write(f"[Gauge {name.replace(' ','')}]\n")
        f.write(f"LON={lon}\n")
        f.write(f"LAT={lat}\n\n")
    
    f.write("[Basin Jakarta]\n")
    for feature in geojson["features"]:
        name = feature["properties"]["name"]
        f.write(f"GAUGE={name.replace(' ','')}\n")
        

print(f"Konversi selesai, disimpan di {output_file}")


Konversi selesai, disimpan di gauges_jakarta.ini
